# Ship or Iceberg
https://www.kaggle.com/c/statoil-iceberg-classifier-challenge

In [40]:
import torch
from torch.autograd import Variable
import torch.nn as nn

import numpy as np
import pandas as pd

# Data

In [36]:
datadir = "/home/christopher/Data/data/ml/ship-or-iceberg/"

with open(datadir + "train.json") as f:
    train = pd.read_json(f).set_index("id")

In [39]:
train.head()
print(np.sqrt(len(train.band_1[0])))

75.0


# Neural Network

In [41]:
# For now lets just build a network to take a single image.
# input - 75 x 75

# Go back to learning nb and play with some convs

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
    
    def forward(self, x):
        pass